# **Méthode naïve**
La méthode naïve repose sur l'exploration de toutes les solutions possibles, partant d'un sommet de départ ` x `.
L'amélioration apportée est l'application de Branch and Bound sur cette solution pour apporter une optimisation remarquable, qui aura comme principes :





# ***une approche itératif de l'algorithme Branch and Bound pour résoudre le problème du voyageur de commerce (PVC)***

# **Description :**
 L'algorithme Branch and Bound pour le TSP explore de manière efficace l'espace de recherche des chemins possibles. Il utilise une file de priorité pour organiser l'exploration en se concentrant d'abord sur les chemins les plus prometteurs, évalués par une borne inférieure du coût total. Cette approche permet d'éliminer de nombreuses branches de l'arbre de recherche qui ne mèneraient pas à une solution optimale, réduisant ainsi le temps de calcul nécessaire pour trouver la meilleure solution.

texte en gras
# **Algorithme :**

**Initialisation :** On commence avec un chemin partiel contenant uniquement le sommet de départ et une borne inférieure de coût basée sur ce chemin.

**Boucle principale :** Tant que la file de priorité n'est pas vide, effectuez les étapes suivantes :
- Extrayez le nœud avec la plus petite borne inférieure.
- Si la borne inférieure du nœud est inférieure au meilleur coût trouvé jusqu'à présent, explorez ce nœud.
- Pour chaque sommet non inclus dans le chemin partiel du nœud, créez un nouveau chemin en ajoutant ce sommet.
- Si le nouveau chemin contient tous les sommets, calculez son coût total et mettez à jour le meilleur chemin et le meilleur coût si ce chemin est meilleur.
- Sinon, calculez la borne inférieure pour le nouveau chemin. Si cette borne est inférieure au meilleur coût, ajoutez le nouveau chemin à la file de priorité pour une exploration ultérieure.

**Terminaison :** L'algorithme se termine lorsque la file de priorité est vide. Le meilleur chemin et le meilleur coût trouvés sont retournés comme solution.

In [7]:
import heapq  # Importe le module heapq pour utiliser la file de priorité
import time  # Importe le module time pour mesurer le temps d'exécution

class Node:
    """
    Classe représentant un nœud dans l'arbre de recherche de l'algorithme Branch and Bound.
    Chaque nœud contient un chemin partiel et une estimation de la borne inférieure du coût total du chemin.
    """
    def __init__(self, path, bound):
        self.path = path  # Chemin partiel: liste des indices des sommets visités
        self.bound = bound  # Borne inférieure: estimation du coût minimum pour compléter le chemin

    def __lt__(self, other):
        # Définit la comparaison de deux nœuds basée sur leur borne inférieure
        # Utilisé par heapq pour organiser la file de priorité
        return self.bound < other.bound

def calculer_borne_inferieure(path, adj_matrix):
    """
    Calcule et retourne la borne inférieure du coût pour un chemin partiel.

    Arguments:
    path -- chemin partiel actuel comme liste d'indices de sommets
    adj_matrix -- matrice d'adjacence représentant les coûts entre les sommets

    La fonction itère sur tous les sommets non inclus dans le chemin partiel et
    ajoute le coût de l'arête la moins chère connectée à chaque sommet non visité.
    """
    bound = 0
    used_vertices = set(path)  # Ensemble des indices des sommets déjà visités

    for i in range(len(adj_matrix)):
        if i not in used_vertices:
            # Filtrer pour exclure les arêtes vers les sommets déjà visités ou des boucles
            available_edges = [adj_matrix[i][j] for j in range(len(adj_matrix)) if j not in used_vertices and i != j]
            if available_edges:
                # Ajoute le coût minimum des arêtes disponibles pour ce sommet
                bound += min(available_edges)

    return bound

def trouver_chemin_hamiltonien(adj_matrix, sommet_depart):
    """
    Trouve le chemin Hamiltonien de coût minimal en utilisant l'algorithme Branch and Bound.

    Arguments:
    adj_matrix -- matrice d'adjacence représentant les coûts entre les sommets
    sommet_depart -- indice du sommet de départ pour le chemin Hamiltonien

    Retourne le meilleur chemin trouvé et son coût.
    """
    # Initialisation du meilleur coût à l'infini pour s'assurer que tout premier chemin trouvé sera pris en compte
    meilleur_cout = float('inf')
    # Initialisation du meilleur chemin comme une liste vide
    meilleur_chemin = []

    # Crée une file de priorité avec le nœud initial représentant le sommet de départ
    NA = [Node(path=[sommet_depart], bound=calculer_borne_inferieure([sommet_depart], adj_matrix))]

    # Boucle tant qu'il y a des nœuds à explorer dans la file de priorité
    while NA:
        # Extrait le nœud avec la plus petite borne inférieure
        n = heapq.heappop(NA)

        # Vérifie si la borne inférieure du nœud est inférieure au meilleur coût trouvé jusqu'à présent
        if n.bound < meilleur_cout:
            # Explore tous les sommets possibles pour continuer le chemin
            for next_vertex in range(len(adj_matrix)):
                # Vérifie si le sommet n'est pas déjà dans le chemin
                if next_vertex not in n.path:
                    # Crée un nouveau chemin en ajoutant le sommet exploré
                    new_path = n.path + [next_vertex]

                    # Vérifie si tous les sommets ont été visités pour former un chemin Hamiltonien
                    if len(new_path) == len(adj_matrix):
                        # Assurez-vous qu'il existe une arête pour retourner au sommet de départ
                        if adj_matrix[new_path[-1]][sommet_depart] != 0:
                            # Calcule le coût total du chemin en incluant le retour au sommet de départ
                            new_cost = sum(adj_matrix[new_path[i]][new_path[i+1]] for i in range(len(new_path)-1)) + adj_matrix[new_path[-1]][sommet_depart]
                            # Si le coût total est inférieur au meilleur coût trouvé, met à jour le meilleur coût et le meilleur chemin
                            if new_cost < meilleur_cout:
                                meilleur_cout = new_cost
                                meilleur_chemin = new_path + [sommet_depart]  # Ajoute le sommet de départ à la fin pour fermer le cycle
                    else:
                        # Calcule la borne inférieure pour le nouveau chemin partiel
                        new_bound = calculer_borne_inferieure(new_path, adj_matrix) + sum(adj_matrix[new_path[i]][new_path[i+1]] for i in range(len(new_path)-1))
                        # Si la nouvelle borne inférieure est prometteuse, ajoute le nouveau nœud à la file de priorité pour exploration future
                        if new_bound < meilleur_cout:
                            heapq.heappush(NA, Node(new_path, new_bound))

    # Retourne le meilleur chemin trouvé et son coût total
    return meilleur_chemin, meilleur_cout

adjacency_matrix  = [
    [0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
    [10, 0, 25, 35, 15, 20, 30, 5, 40, 35, 45, 50, 55, 60, 65, 70],
    [15, 25, 0, 5, 10, 30, 40, 15, 25, 30, 20, 45, 30, 35, 40, 50],
    [20, 35, 5, 0, 40, 25, 15, 10, 20, 45, 30, 35, 40, 50, 55, 60],
    [25, 15, 10, 40, 0, 20, 10, 25, 30, 35, 50, 55, 60, 65, 70, 75],
    [30, 20, 30, 25, 20, 0, 35, 30, 5, 15, 40, 45, 50, 55, 60, 65],
    [35, 30, 40, 15, 10, 35, 0, 20, 25, 30, 45, 60, 65, 70, 75, 80],
    [40, 5, 15, 10, 25, 30, 20, 0, 35, 40, 55, 35, 40, 45, 50, 55],
    [45, 40, 25, 20, 30, 5, 25, 35, 0, 15, 10, 25, 30, 35, 40, 45],
    [50, 35, 30, 45, 35, 15, 30, 40, 15, 0, 25, 20, 15, 20, 25, 30],
    [55, 45, 20, 30, 50, 40, 45, 55, 10, 25, 0, 15, 30, 35, 40, 45],
    [60, 50, 45, 35, 55, 45, 60, 35, 25, 20, 15, 0, 10, 15, 20, 25],
    [65, 55, 30, 40, 60, 50, 65, 40, 30, 15, 30, 10, 0, 25, 30, 35],
    [70, 60, 35, 50, 65, 55, 70, 45, 35, 20, 35, 15, 25, 0, 10, 15],
    [75, 65, 40, 55, 70, 60, 75, 50, 40, 25, 40, 20, 30, 10, 0, 5],
    [80, 70, 50, 60, 75, 65, 80, 55, 45, 30, 45, 25, 35, 15, 5, 0]
]

# Exécution de l'algorithme et mesure du temps
sommet_depart = 0  # Définit le sommet de départ
start_time = time.time()  # Marque le début de l'exécution
meilleur_chemin, meilleur_cout = trouver_chemin_hamiltonien(adjacency_matrix, sommet_depart)  # Exécute l'algorithme
end_time = time.time()  # Marque la fin de l'exécution

# Affiche les résultats
print("Le meilleur chemin trouvé est :", meilleur_chemin)
print("Avec un coût de :", meilleur_cout)
print(f"Temps d'exécution: {end_time - start_time} secondes")


Le meilleur chemin trouvé est : [0, 1, 7, 3, 2, 4, 6, 9, 13, 14, 15, 12, 11, 10, 8, 5, 0]
Avec un coût de : 220
Temps d'exécution: 51.35639548301697 secondes


# ***une approche récursive de l'algorithme Branch and Bound pour résoudre le problème du voyageur de commerce (PVC)***

---
# **Description:**
L'algorithme optimise la recherche du chemin de coût minimal dans le Problème du Voyageur de Commerce en calculant d'abord une borne inférieure initiale avec les coûts minimaux des arêtes. Il explore ensuite récursivement tous les chemins potentiels, en ajoutant sélectivement des sommets non visités, tout en mettant à jour le coût et la borne inférieure pour chaque chemin partiel. Si un chemin partiel est jugé prometteur (coût partiel plus borne inférieure inférieur au meilleur coût actuel), l'exploration continue; sinon, l'algorithme élimine ce chemin de l'espace de recherche. Cette méthode réduit l'espace de recherche plus efficacement que l'exploration exhaustive, en se concentrant uniquement sur les chemins les plus prometteurs.

# **Algorithme:**
**Initialisation :** Démarre avec un chemin initial contenant uniquement le sommet de départ et une borne inférieure calculée à partir des coûts minimum et second minimum de tous les sommets.

**Fonction Récurrente TSP_rec :**

Si tous les sommets ont été visités, vérifie la formation d'un cycle valide et met à jour le coût et le chemin si c'est une meilleure solution.
Pour chaque sommet non visité, ajoute le sommet au chemin courant et calcule le nouveau coût et la nouvelle borne.
Si la nouvelle borne plus le coût courant est inférieure au meilleur coût trouvé, continue l'exploration récursive avec ce nouveau chemin.
Réinitialise l'état pour essayer un nouveau sommet comme successeur du sommet courant.

**Terminaison :** Lorsque tous les chemins possibles ont été explorés, le chemin et le coût minimaux sont retournés.

In [ ]:
import time  # Importation du module time pour calculer le temps d'exécution
import math  # Importation du module math pour utiliser la fonction ceil

# Fonction pour copier le chemin courant dans le chemin final et ajouter le premier sommet à la fin pour former un cycle
def copy_to_final(curr_path):
    final_path[:N + 1] = curr_path[:]  # Copie le chemin courant dans final_path
    final_path[N] = curr_path[0]  # Ajoute le premier sommet à la fin pour compléter le cycle

# Fonction pour trouver le coût minimum d'une arête sortante du sommet i
def first_min(adj, i):
    # Retourne le coût minimum parmi tous les arcs sortants du sommet i, à l'exception de l'arc vers lui-même
    return min(adj[i][j] for j in range(N) if i != j)

# Fonction pour trouver le deuxième coût minimum d'une arête sortante du sommet i
def second_min(adj, i):
    values = [adj[i][j] for j in range(N) if i != j]  # Liste des coûts des arêtes sortantes du sommet i
    values.sort()  # Trie la liste des coûts
    return values[1]  # Retourne le deuxième coût le plus bas

# Fonction récursive principale de l'algorithme Branch and Bound
def TSP_rec(adj, curr_bound, curr_weight, level, curr_path, visited):
    global final_res  # Référence à la variable globale pour le résultat final
    # Si tous les sommets ont été visités, vérifie si le chemin forme un cycle valide
    if level == N:
        if adj[curr_path[level - 1]][curr_path[0]] != 0:  # Vérifie s'il y a un retour au point de départ
            curr_res = curr_weight + adj[curr_path[level - 1]][curr_path[0]]  # Calcule le coût total du cycle
            if curr_res < final_res:  # Si le coût total est inférieur au meilleur coût trouvé jusqu'à présent
                copy_to_final(curr_path)  # Copie le chemin courant dans le chemin final
                final_res = curr_res  # Met à jour le meilleur coût
        return

    # Boucle pour explorer les successeurs du nœud courant
    for i in range(N):
        if adj[curr_path[level - 1]][i] != 0 and not visited[i]:  # Si le sommet i est accessible et non visité
            temp = curr_bound  # Sauvegarde la borne courante
            curr_weight += adj[curr_path[level - 1]][i]  # Ajoute le coût de l'arête au poids courant

            # Ajuste la borne pour le niveau suivant
            if level == 1:
                curr_bound -= (first_min(adj, curr_path[level - 1]) + first_min(adj, i)) / 2
            else:
                curr_bound -= (second_min(adj, curr_path[level - 1]) + first_min(adj, i)) / 2

            # Si la borne inférieure actuelle + le poids courant est inférieure au meilleur résultat
            if curr_bound + curr_weight < final_res:
                curr_path[level] = i  # Ajoute le sommet i au chemin courant
                visited[i] = True  # Marque le sommet i comme visité

                # Appel récursif pour explorer le niveau suivant
                TSP_rec(adj, curr_bound, curr_weight, level + 1, curr_path, visited)

            # Réinitialise le poids courant et la borne pour explorer d'autres branches
            curr_weight -= adj[curr_path[level - 1]][i]
            curr_bound = temp

            # Réinitialise le tableau des sommets visités pour le prochain appel
            visited = [False] * len(visited)
            for j in range(level):
                if curr_path[j] != -1:
                    visited[curr_path[j]] = True

# Fonction pour initialiser et exécuter l'algorithme TSP
def TSP(adj):
    curr_bound = 0  # Borne initiale
    curr_path = [-1] * (N + 1)  # Chemin courant initialisé à -1
    visited = [False] * N  # Tableau des sommets visités initialisé à False

    # Calcule la borne initiale
    for i in range(N):
        curr_bound += (first_min(adj, i) + second_min(adj, i))
    curr_bound = math.ceil(curr_bound / 2)  # Arrondit la borne à l'entier supérieur

    visited[0] = True  # Marque le premier sommet comme visité
    curr_path[0] = 0  # Commence le chemin par le premier sommet

    # Appelle la fonction récursive pour commencer l'exploration
    TSP_rec(adj, curr_bound, 0, 1, curr_path, visited)

# Code principal
if __name__ == "__main__":
    # Matrice d'adjacence représentant le graphe
  '''  adjacency_matrix  =[
    [0, 4, 7, 2, 5, 4],
    [4, 0, 3, 2, 1, 2],
    [7, 3, 0, 2, 6, 3],
    [2, 2, 2, 0, 5, 3],
    [5, 1, 6, 5, 0, 2],
    [4, 2, 3, 3, 2, 0] ]'''

    N = len(adjacency_matrix)  # Nombre de sommets dans le graphe
    final_path = [None] * (N + 1)  # Chemin final
    visited = [False] * N  # Sommets visités
    final_res = float('inf')  # Meilleur résultat initialisé à l'infini

    start_time = time.time()  # Début du chronométrage
    TSP(adjacency_matrix)  # Exécution de l'algorithme TSP
    end_time = time.time()  # Fin du chronométrage

    # Affichage des résultats
    print(f"Minimum cost: {final_res}")
    print(f"Path Taken: {final_path}")
    print(f"Time taken: {end_time - start_time} seconds")


Minimum cost: 14
Path Taken: [0, 1, 4, 5, 2, 3]
Time taken: 0.0021572113037109375 seconds
